In [1]:
import os
import csv
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split

In [2]:
parent = 'Milestone/fnc-1-baseline-master/fnc-1/'

In [4]:
train_bodies = pd.read_csv(parent + 'train_bodies.csv')
train_bodies.head(10)

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...
5,8,There is so much fake stuff on the Internet in...
6,9,"(CNN) -- A meteorite crashed down in Managua, ..."
7,10,"Move over, Netflix and Hulu.\nWord has it that..."
8,11,We’ve all seen the traditional depictions of G...
9,13,A SOLDIER has been shot at Canada’s National W...


In [5]:
test_bodies = pd.read_csv(parent + 'competition_test_bodies.csv')
test_bodies.head(10)

,Body ID,articleBody
0,1,Al-Sisi has denied Israeli reports stating tha...
1,2,A bereaved Afghan mother took revenge on the T...
2,3,CNBC is reporting Tesla has chosen Nevada as t...
3,12,A 4-inch version of the iPhone 6 is said to be...
4,19,GR editor’s Note\n\nThere are no reports in th...
5,20,“I once sent Tim an email about the quality of...
6,21,Absolutely no one was devastated when news bro...
7,28,In designing its first wearable device in the ...
8,31,"The picture of a possible ""Crabzilla"" measurin..."
9,33,"Early Tuesday morning, Dornella Conner was “ri..."


In [6]:
# load training data
train_df = pd.read_csv(parent + '/train_stances.csv')
# load testing data
test_df = pd.read_csv(parent + '/competition_test_stances.csv')

In [7]:
train_df.replace('agree',0,True)
train_df.replace('disagree',1,True)
train_df.replace('discuss',2,True)
train_df.replace('unrelated',3,True)

test_df.replace('agree',0,True)
test_df.replace('disagree',1,True)
test_df.replace('discuss',2,True)
test_df.replace('unrelated',3,True)

In [8]:
combined_df_train = train_df.join(train_bodies.set_index('Body ID'), on='Body ID')
combined_df_test = test_df.join(test_bodies.set_index('Body ID'), on='Body ID')

In [10]:
combined_df_train

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,3,Danny Boyle is directing the untitled film\n\n...
1,Hundreds of Palestinians flee floods in Gaza a...,158,0,Hundreds of Palestinians were evacuated from t...
2,"Christian Bale passes on role of Steve Jobs, a...",137,3,30-year-old Moscow resident was hospitalized w...
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,3,(Reuters) - A Canadian soldier was shot at the...
4,Spider burrowed through tourist's stomach and ...,1923,1,"Fear not arachnophobes, the story of Bunbury's..."
...,...,...,...,...
49967,Urgent: The Leader of ISIL 'Abu Bakr al-Baghda...,1681,3,(CNN) -- Boko Haram laughed off Nigeria's anno...
49968,Brian Williams slams social media for speculat...,2419,3,Along with unveiling the Apple Watch earlier t...
49969,Mexico Says Missing Students Not Found In Firs...,1156,0,The bodies found in a mass grave were confirme...
49970,US Lawmaker: Ten ISIS Fighters Have Been Appre...,1012,2,Rep. Duncan Hunter (R-CA) told Greta Van Suste...


In [11]:
combined_df_test

,Headline,Body ID,Stance,articleBody
0,Ferguson riots: Pregnant woman loses eye after...,2008,3,A RESPECTED senior French police officer inves...
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,3,Dave Morin's social networking company Path is...
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,3,A bereaved Afghan mother took revenge on the T...
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",1793,3,Hewlett-Packard is officially splitting in two...
4,Argentina's President Adopts Boy to End Werewo...,37,3,An airline passenger headed to Dallas was remo...
...,...,...,...,...
25408,The success of the Affordable Care Act is a hu...,2582,0,"Congressional Republicans, evidently hoping th..."
25409,The success of the Affordable Care Act is a hu...,2583,2,Did Obamacare work?\n\nIt’s worth reflecting u...
25410,The success of the Affordable Care Act is a hu...,2584,1,Millions may lose coverage next year if Congre...
25411,The success of the Affordable Care Act is a hu...,2585,1,"Come November, the grim trudge across the incr..."


In [174]:
headlines = list()

for i in combined_df_train['Headline']:
    if i not in headlines:
        headlines.append(i)

In [175]:
len(headlines)

1648

In [12]:
temp2 = combined_df_train.loc[combined_df_train['Stance'] == 3]
temp2.shape

(36545, 4)

temp2.shape

In [177]:
df = pd.DataFrame()
cnt, sum1 = 0, 0

for i in headlines:
    temp1 = combined_df_train.loc[combined_df_train['Headline'] == i]
    related = temp1.loc[(temp1['Stance'] == 0) | (temp1['Stance'] == 1) | (temp1['Stance'] == 2)]
    df_t = df.append(related)
    df = df_t
    unrelated = temp1.loc[temp1['Stance'] == 3]
    if related.shape[0] < 1:
        cnt = cnt + 1
        sum1 = sum1 + unrelated.shape[0]
        continue
    if unrelated.shape[0] < related.shape[0]:
        df_t = df.append(unrelated)
    else:
        df_t = df.append(unrelated.iloc[:int(related.shape[0])])
    df = df_t

print(df.shape, cnt)

(26854, 4) 0


In [178]:
df_t = df.append(df1)
df = df_t.append(df2.iloc[:df1.shape[0]])

In [179]:
df.shape

(26854, 4)

In [180]:
temp2 = df[df['Stance'] == 3]
temp2.shape

(13427, 4)

In [159]:
stances = df['Stance']
stances.replace(0, 'agree',True)
stances.replace(1, 'disagree',True)
stances.replace(2, 'discuss',True)
stances.replace(3, 'unrelated',True)

In [ ]:
df['Stance'] = stances

In [181]:
df = df.sample(frac = 1)

In [182]:
df.to_csv('full_train.csv', index=False, encoding='utf-8')
df

,Headline,Body ID,Stance,articleBody
37271,"Boko Haram agrees to ceasefire, release of sch...",2403,2,More than 200 missing schoolgirls kidnapped by...
347,Small red book among finds in 113-year-old Bos...,690,3,"Apple’s next-generation MacBook Air, which is ..."
42049,Missing jetliners raise fear of new 9/11-style...,1312,0,Nouakchott — The Tripoli airport siege led to ...
7498,Conde Nast's rat problem at One World Trade Ce...,387,3,"(Nov. 26, 2014) – Your Thanksgiving turkey is ..."
14331,WFB’s Bill Gertz: Obama Administration ‘Trying...,2010,2,Islamist militias in Libya took control of nea...
...,...,...,...,...
3269,Give a homeless man $100 and see what he does ...,2421,3,Apple has been evaluating platinum for future ...
34356,"James Wright Foley, Kidnapped Journalist, Appa...",1385,3,A DESPERATE mother was tricked into feasting o...
48724,Macaulay Culkin Dead: Internet Death Hoax Clai...,1555,0,"Once again, rumors that Macaulay Culkin died h..."
39369,Report claims Kim Jong-un underwent surgery to...,2417,2,So it wasn’t just “discomfort” that’s been tro...


In [162]:
stances1 = combined_df_test['Stance']
stances1.replace(0, 'agree',True)
stances1.replace(1, 'disagree',True)
stances1.replace(2, 'discuss',True)
stances1.replace(3, 'unrelated',True)
combined_df_test['Stance'] = stances1

In [183]:
combined_df_test

,Headline,Body ID,Stance,articleBody
0,Ferguson riots: Pregnant woman loses eye after...,2008,3,A RESPECTED senior French police officer inves...
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,3,Dave Morin's social networking company Path is...
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,3,A bereaved Afghan mother took revenge on the T...
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",1793,3,Hewlett-Packard is officially splitting in two...
4,Argentina's President Adopts Boy to End Werewo...,37,3,An airline passenger headed to Dallas was remo...
...,...,...,...,...
25408,The success of the Affordable Care Act is a hu...,2582,0,"Congressional Republicans, evidently hoping th..."
25409,The success of the Affordable Care Act is a hu...,2583,2,Did Obamacare work?\n\nIt’s worth reflecting u...
25410,The success of the Affordable Care Act is a hu...,2584,1,Millions may lose coverage next year if Congre...
25411,The success of the Affordable Care Act is a hu...,2585,1,"Come November, the grim trudge across the incr..."


In [184]:
combined_df_test.to_csv('full_test.csv', index=False, encoding='utf-8')

In [60]:
train_final = pd.DataFrame(columns = ['text_a', 'text_b', 'labels'])
test_final = pd.DataFrame(columns = ['text_a', 'text_b', 'Body ID', 'label'])

In [61]:
train_final

,text_a,text_b,labels


In [62]:
import re

def returnString(inp):
    ans = list()
    for i in inp:
        # i = i.strip().lower()
        i = re.sub(r"\n", ' ', i)
        ans.append(i)
    return ans

In [63]:
train_head = returnString(combined_df_train['Headline'])
train_body = returnString(combined_df_train['articleBody'])

In [64]:
# train_final['text_a'] = combined_df_train['Headline']
# train_final['text_b'] = combined_df_train['articleBody']

train_final['text_a'] = train_head
train_final['text_b'] = train_body
train_final['labels'] = combined_df_train['Stance'].astype(int)
train_final

,text_a,text_b,labels
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film Se...,3
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...,0
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...,3
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...,3
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's...",1
...,...,...,...
49967,Urgent: The Leader of ISIL 'Abu Bakr al-Baghda...,(CNN) -- Boko Haram laughed off Nigeria's anno...,3
49968,Brian Williams slams social media for speculat...,Along with unveiling the Apple Watch earlier t...,3
49969,Mexico Says Missing Students Not Found In Firs...,The bodies found in a mass grave were confirme...,0
49970,US Lawmaker: Ten ISIS Fighters Have Been Appre...,Rep. Duncan Hunter (R-CA) told Greta Van Suste...,2


In [67]:
train_final.iloc[0][1]

'Danny Boyle is directing the untitled film  Seth Rogen is being eyed to play Apple co-founder Steve Wozniak in Sony’s Steve Jobs biopic.  Danny Boyle is directing the untitled film, based on Walter Isaacson\'s book and adapted by Aaron Sorkin, which is one of the most anticipated biopics in recent years.  Negotiations have not yet begun, and it’s not even clear if Rogen has an official offer, but the producers — Scott Rudin, Guymon Casady and Mark Gordon — have set their sights on the talent and are in talks.  Of course, this may all be for naught as Christian Bale, the actor who is to play Jobs, is still in the midst of closing his deal. Sources say that dealmaking process is in a sensitive stage.  Insiders say Boyle will is flying to Los Angeles to meet with actress to play one of the female leads, an assistant to Jobs. Insiders say that Jessica Chastain is one of the actresses on the meeting list.  Wozniak, known as "Woz," co-founded Apple with Jobs and Ronald Wayne. He first met J

In [68]:
test_head = returnString(combined_df_test['Headline'])
test_body = returnString(combined_df_test['articleBody'])

In [69]:
# test_final['text_a'] = combined_df_test['Headline']
# test_final['text_b'] = combined_df_test['articleBody']

test_final['Headline'] = combined_df_test['Headline']
test_final['text_a'] = test_head
test_final['text_b'] = test_body
test_final['label'] = combined_df_test['Stance'].astype(int)
test_final['Body ID'] = combined_df_test['Body ID'].astype(int)
test_final

,text_a,text_b,Body ID,label,Headline
0,Ferguson riots: Pregnant woman loses eye after...,A RESPECTED senior French police officer inves...,2008,3,Ferguson riots: Pregnant woman loses eye after...
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,Dave Morin's social networking company Path is...,1550,3,Crazy Conservatives Are Sure a Gitmo Detainee ...
2,A Russian Guy Says His Justin Bieber Ringtone ...,A bereaved Afghan mother took revenge on the T...,2,3,A Russian Guy Says His Justin Bieber Ringtone ...
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",Hewlett-Packard is officially splitting in two...,1793,3,"Zombie Cat: Buried Kitty Believed Dead, Meows ..."
4,Argentina's President Adopts Boy to End Werewo...,An airline passenger headed to Dallas was remo...,37,3,Argentina's President Adopts Boy to End Werewo...
...,...,...,...,...,...
25408,The success of the Affordable Care Act is a hu...,"Congressional Republicans, evidently hoping th...",2582,0,The success of the Affordable Care Act is a hu...
25409,The success of the Affordable Care Act is a hu...,Did Obamacare work? It’s worth reflecting upo...,2583,2,The success of the Affordable Care Act is a hu...
25410,The success of the Affordable Care Act is a hu...,Millions may lose coverage next year if Congre...,2584,1,The success of the Affordable Care Act is a hu...
25411,The success of the Affordable Care Act is a hu...,"Come November, the grim trudge across the incr...",2585,1,The success of the Affordable Care Act is a hu...


In [27]:
train, val = train_test_split(train_final, random_state=42, test_size=0.10)

In [28]:
train

,text_heading,text_articles,labels
44610,'There is no limit to their depravity' Islamic...,A British man who went to fight against Islami...,2
10499,Amazon.com to open first physical store in Man...,A short video clip has been circulating online...,3
43957,ISIS fighters 'have contracted Ebola': World H...,Militants from Islamic State (Isis) claimed to...,3
49559,IS militant 'Jihadi John' named as Mohammed Em...,North Korea's Kim Jong-un has piled on so much...,3
1851,Sugarhill Gang Rapper ‘Big Bank Hank’ Dies at 57,"On the same day Michael Brown was buried, a Fe...",3
...,...,...,...
11284,Boko Haram Denies Nigeria Cease-Fire Claim,Jose Canseco was injured in an accidental shoo...,3
44732,DNA tests prove Lebanon IS holding ISIS chief ...,BEIRUT — Lebanon’s military has detained a wif...,2
38158,"Sorry, Fido. Pope Francis did NOT say our pets...",If you're already using iOS 8's iCloud Drive f...,3
860,Is Kim Jong-un’s sister now the power behind t...,Arrived in the convent of Macerata in June las...,3


In [29]:
val

,text_heading,text_articles,labels
17851,‘Photo of slain IS leader’ reported to be doct...,(IraqiNews.com) Thousands of social media user...,2
6400,Report: A Radically Redesigned 12-Inch MacBook...,"On Monday morning, a report surfaced claiming ...",3
6653,iOS 8 bug could delete your iCloud Drive docum...,Updated at 11:45 p.m. ET\n\nThere was an unexp...,3
44559,"Boko Haram Could Release Kidnapped Girls, Nige...",Militant group Boko Haram is reportedly holdin...,2
37749,ISIS Video Shows Execution of David Cawthorne ...,WARNING: GRAPHIC IMAGES. A masked militant cla...,3
...,...,...,...
7029,What Glenn Beck Fears May Be Done With the 11 ...,Forget sweater weather and crisp autumn leaves...,3
33047,The Internet Tried To Make Axl Rose Its Latest...,Those caught with the same moniker as the dict...,3
44208,Meteorite 'caused mysterious boom in Nicaragua',Islamic State militants and sympathisers are t...,3
14163,Banksy 'Arrested & Real Identity Revealed' Is ...,Wanted: Islamic State group chief Abu Bakr al-...,3


In [72]:
train_final.to_csv('train.csv', index=False, encoding='utf-8')

In [73]:
test_final.to_csv('test.csv', index=False, encoding='utf-8')